In [1]:
import pandas as pd
import numpy as np

def gde(df_raw):
    df = df_raw.fillna(0).copy()  # Preenche NaN com 0

    # A primeira coluna contém os nomes dos danos
    col_dano = df.columns[0]
    nomes_danos = df[col_dano].astype(str)

    # Identificar as peças com base no primeiro nível do MultiIndex (nível 0)
    pecas = df.columns.levels[0][1:]  # Ignorar a primeira coluna ('Danos')

    registros = []

    for i in range(len(df)):
        dano = str(df.iloc[i, 0]).strip()
        if dano.lower() == 'danos' or dano == '':
            continue

        for peca in pecas:
            try:
                fi = int(df[(peca, 'Fi')].iloc[i]) if pd.notna(df[(peca, 'Fi')].iloc[i]) else 0
                fp = int(df[(peca, 'Fp')].iloc[i]) if pd.notna(df[(peca, 'Fp')].iloc[i]) else 0
            except KeyError:
                fi, fp = 0, 0

            # Cálculo do dano "d"
            if fi <= 2.0:
                d = 0.8 * fi * fp
            elif fi >= 3.0:
                d = (12 * fi - 28) * fp
            else:
                d = 0

            registros.append({
                'peça': peca,
                'dano': dano,
                'fi': fi,
                'fp': fp,
                'd': d
            })

    # Criar DataFrame final
    df_resultado = pd.DataFrame(registros)

    # Criar DataFrames separados por peça
    dfs_por_peca = {
        peca: df_resultado[df_resultado['peça'] == peca].reset_index(drop=True)
        for peca in df_resultado['peça'].unique()
    }

    return df_resultado, dfs_por_peca


# Nova função para calcular GDE e GDF por peça
def calcular_gde_por_peca(dfs_por_peca):
    resultados_gde = {}

    for peca, df in dfs_por_peca.items():
        d_values = df['d'].values

        if len(d_values) == 0 or d_values.sum() == 0:
            # Se não há danos, valores nulos
            resultados_gde[peca] = {
                'gde': 0,
                'gde_max': 0,
                'gde_total': 0,
                'gdf': 0
            }
            continue

        d_total = d_values.sum()
        d_max = d_values.max()

        # Cálculo do GDE
        gde = d_max * (1 + ((d_total - d_max) / d_total))
        gde_values = [gde]
        gde_max = max(gde_values)
        gde_total = sum(gde_values)

        print(f"Peça: {peca}, GDE: {gde}, GDE Max: {gde_max}, GDE Total: {gde_total}")
        # Cálculo do GDF
        gdf = gde_max * np.sqrt(1 + gde_total - gde_max) / gde_total

        resultados_gde[peca] = {
            'gde': gde,
            'gde_max': gde_max,
            'gde_total': gde_total,
            'gdf': gdf
        }

    return resultados_gde


In [2]:
# pd.set_option('display.max_rows', None)
df = pd.read_excel('inspecao_vigas.xlsx', header=[0, 1])
df_resultado, df_preca = gde(df)
df_preca['Viga lateral (jusante)']


,peça,dano,fi,fp,d
0,Viga lateral (jusante),Carbonatação,0,0,0.0
1,Viga lateral (jusante),Cobrimento Deficiente,0,0,0.0
2,Viga lateral (jusante),Contaminação por Cloretos,0,0,0.0
3,Viga lateral (jusante),Corrosão de armaduras,0,0,0.0
4,Viga lateral (jusante),Danos por impacto,0,0,0.0
5,Viga lateral (jusante),Desagregação,0,0,0.0
6,Viga lateral (jusante),Desplacamento,0,0,0.0
7,Viga lateral (jusante),Desvio de Geometria,0,0,0.0
8,Viga lateral (jusante),Eflorescência,0,0,0.0
9,Viga lateral (jusante),Falha de Concretagem,1,3,2.4


In [3]:
# Calcular GDE por peça
gde_resultados = calcular_gde_por_peca(df_preca)

# Exibir resultado
for peca, res in gde_resultados.items():
    print(f"\nPeça: {peca}")
    print(f"GDE: {res['gde']:.2f}")
    print(f"GDF: {res['gdf']:.2f}")

Peça: Viga lateral (jusante), GDE: 3.6000000000000005, GDE Max: 3.6000000000000005, GDE Total: 3.6000000000000005
Peça: Viga lateral (montante), GDE: 3.6000000000000005, GDE Max: 3.6000000000000005, GDE Total: 3.6000000000000005

Peça: Viga lateral (jusante)
GDE: 3.60
GDF: 1.00

Peça: Viga lateral (montante)
GDE: 3.60
GDF: 1.00
